In [1]:

import numpy as np
import pandas as pd

matrix = np.array([
    [0.156, 0.075, 0.099, 0.101],
    [0.081, 0.072, 0.081, 0.093],
    [0.055, 0.086, 0.067, 0.090],
    [0.055, 0.070, 0.080, 0.080],
    [0.067, 0.081, 0.238, 0.071],
    [0.243, 0.057, 0.117, 0.062],
    [0.032, 0.128, 0.017, 0.131],
    [0.023, 0.088, 0.022, 0.134],
    [0.030, 0.119, 0.112, 0.095],
    [0.081, 0.092, 0.125, 0.081],
    [0.011, 0.131, 0.042, 0.063]
])

criteria = ['C1', 'C2', 'C3', 'C4']
sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10', 'Site11']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.16376103, 0.29605439, 0.46393757, 0.076247])

# V-shape preference thresholds (can be tuned)
p_list = [2.998, 1.441, 0.783, 3.957]

def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

def vshape_preference(d, p):
    return np.clip(d / p, 0, 1)

def calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list):
    n = norm_matrix.shape[0]
    m = norm_matrix.shape[1]
    preference_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            pref_sum = 0
            for k in range(m):
                if criteria_types[k] == 'max':
                    diff = norm_matrix[i, k] - norm_matrix[j, k]
                else:  # 'min'
                    diff = norm_matrix[j, k] - norm_matrix[i, k]
                pref = vshape_preference(diff, p_list[k])
                pref_sum += weights[k] * pref
            preference_matrix[i, j] = pref_sum
    return preference_matrix

def calculate_flows(preference_matrix):
    n = preference_matrix.shape[0]
    phi_plus = preference_matrix.sum(axis=1) / (n - 1)
    phi_minus = preference_matrix.sum(axis=0) / (n - 1)
    net_flows = phi_plus - phi_minus
    return phi_plus, phi_minus, net_flows

def promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria):
    norm_matrix = normalize_matrix(matrix, criteria_types)
    preference_matrix = calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list)
    phi_plus, phi_minus, net_flows = calculate_flows(preference_matrix)
    
    # Prepare results DataFrame
    df = pd.DataFrame(matrix, columns=criteria, index=sites)
    df['Leaving Flow'] = phi_plus
    df['Entering Flow'] = phi_minus
    df['Net Flow'] = net_flows
    df['Rank'] = df['Net Flow'].rank(ascending=False, method='min').astype(int)
    df = df.sort_values('Rank')
    return df

# Run PROMETHEE II
result_df = promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria)
print(result_df)


           C1     C2     C3     C4  Leaving Flow  Entering Flow  Net Flow  \
Site5   0.067  0.081  0.238  0.071      0.422217       0.057108  0.365109   
Site9   0.030  0.119  0.112  0.095      0.196663       0.059084  0.137579   
Site10  0.081  0.092  0.125  0.081      0.170970       0.068423  0.102547   
Site6   0.243  0.057  0.117  0.062      0.154830       0.146403  0.008427   
Site1   0.156  0.075  0.099  0.101      0.111073       0.111810 -0.000737   
Site11  0.011  0.131  0.042  0.063      0.134807       0.175207 -0.040401   
Site2   0.081  0.072  0.081  0.093      0.062067       0.146831 -0.084763   
Site3   0.055  0.086  0.067  0.090      0.058404       0.149316 -0.090913   
Site7   0.032  0.128  0.017  0.131      0.126995       0.218127 -0.091132   
Site4   0.055  0.070  0.080  0.080      0.053820       0.158202 -0.104381   
Site8   0.023  0.088  0.022  0.134      0.038436       0.239770 -0.201334   

        Rank  
Site5      1  
Site9      2  
Site10     3  
Site6      4  
